<a href="https://colab.research.google.com/github/ThomasGesseyJones/DARA_21cm_Signal_Simulation/blob/main/DARA_21cm_Signal_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DARA 21-cm Signal Simulation


In this notebook we'll create a simple simulation of the 21-cm signal together. We shall impliment the key physics we learnt in the theory lectures earlier and produce our own predictions for what the 21-cm global signal and power spectrum may look like.

If you haven't already it found it there this notebook is hosted on GitHub at
[https://github.com/ThomasGesseyJones/DARA_21cm_Signal_Simulation](https://github.com/ThomasGesseyJones/DARA_21cm_Signal_Simulation)

This repository also provides a short README.md on how to get your own copy of this notebook on your google account so you can follow along and experiment!

## Types of 21-cm Signal Simulation

Before we start we need to decide what type of 21-cm signal simulation we are going to create. Broadly speaking we can split 21-cm signal simulations into three categories:
- Analytic (e.g.)
- Full-numerical (e.g. )
- Semi-numerical (e.g. )

**Analytic** simulations typically operate by solving the differential equations that describe the 21-cm signal's evolution under the assumption of a homogeneous universe. So they treat the 21-cm signal $T_{\rm 21}$ at a given redshift $z$ and all things it depends upon, such as the kinetic temperature $T_{\rm K}$ and electron fraction $x_{\rm e}$, as homogeneous throughout the entire universe. This has the advantage that for each of theese quantities at each redshift step you only need to store one number, and so it requires very little memory. In addition, because radiative transfer does not need to be modelled solving the differential equaitons is computaitonally easy, leading to very quick codes. Unfortunatley, the 21-cm signal is not uniform and assuming so means you cannot make predictions for the 21-cm power spectrum, and your predicitons of the global 21-cm signal are biased (#TODO ciation). 

**Full-numerical** simulations using particle-hydrodynamics codes to model the evolution of large portions of the universe. Physics often modelled includes, dark matter and baryons collapsing into halos under gravity, chemical and thermal evolution of the gas, star formation, and radiative transfer. As star formation occurs on much smaller scales (parsecs) than the cosmological volumes of interest (Megaparsecs or Gigaparsecs) these simulations require billions of particles to get sufficient resolution. Consequently, they are hugely computationally expensive. For example the CODA3 recently ran on the XX supercomputer taking... (#TODO citations). 

**Semi-numerical** simulations are somewhat of a compromise between the previous two. They aim to be physically realistic enougth to produce accurate predictions of the 21-cm signal, including its spatial variations, but fast and low-memory enougth to be run on a commerically available desktop computer. This is typically achieved by splitting the cosmological volume into simulation cells. Each cell can have a different temperature, ionization fraction, 21-cm signal, etc..., and hence allows for the variation in the signal to be modelled. Physics below the size of the cell, e.g. star formation, is not numerically modelled but instead treated using analytic approximations, while large scale physics that goes between cells, e.g. radiative transfer, is treated numerically. Hence, the name semi-numerical. 

Since, we want accurate enougth predictions in a reasonable time frame (ideally the hour length of this session) we shall develop a simple semi-numerical simulation code. 

##Representing the Universe

As mentioned above we are going to split our simulation of the universe into $N \times N \times N$ pixels each with some sidelength $L_{\rm pix}$. Prior wisdom shows $L_{\rm pix} = 3$ comoving Megaparsecs is a good choice for the latter value. In case, you haven't seen it before comoving lengths are distances that scale with the expansion of the universe. So 3 comoving Megaparsecs means the pixel would be 3 Megaparsecs large today, but it would have been 1.5 Megaparsecs in physical size at redshift $1$, when the universe was half its current size, and only 0.06 Megaparsecs at redshift $49$. We choose to have our cells be defined in comoving Megaparsecs so they, and our simulation box, scales automatically with the expansion of the universe. 

Let us start then by defining these all important quantities as constants for our simulation

In [1]:
L_PIX = 3   # Pixel size in cMpc
N_PIX = 10  # Number of pixels on each side of the simulation cube (kept small so runtime short)

Now we have the space of our simualation sorted, let us think about the time variable. Time is a pain to work with in cosmology in general because its specific to the exact cosmological parameters you are using, and itsn't a direct observable. Instead its more common to work in redshift $z$. Hence that is what we shall do to. For simplicity we shall hardcode our simulation to run from redshift $50$ to $6$ (roughly the end of reionization when $T_{\rm 21}$ vanishes) in steps of size 1

In [6]:
import numpy as np
ZS_SIM = np.arange(50, 5, -1)  # Redshift time-steps of our simulation
print(ZS_SIM)

[50 49 48 47 46 45 44 43 42 41 40 39 38 37 36 35 34 33 32 31 30 29 28 27
 26 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10  9  8  7  6]


## Algorithm Overview

Before we go any further let us think about our end goal so we can keep it in mind as we code. We wish to compute the 21-cm signal $T_{\rm 21}$. As we establihsed in the theory lecture earlier to do this we need to know
- The background radiation temperature $T_{\gamma}$
- The IGM kinetic temperature $T_{\rm K}$
- 
- 
- 

In each cell at each time step, we will need to know its overdensity $\delta$ (the fractional increase in density of this cell compared to the universal average), kinetic temperature $T_{\rm K}$, ionization fraction $x_{\rm e}$, and the strength of the Lyman-$\alpha$ radiation field $J_{\rm \alpha}$ so that we can calculate the 21-cm signal. 

Our simulation needs to start somewhere, for now let us brush over that and just say we have some initial

#Initial Conditions

Our simulation needs to start somewhere, a common choice is around redshift $z = 50$, because our current expectation is that no star formation will have occured before this redshift and that structure formation up to this point can be modelled using linear perturbation theory. Which is a long winded way of saying we think we have a rough idea of what the universe should look like at this point by extrapolating from the CMB. With theory prediciting the universe's density and temperature should be a Gaussian random field, with a fairly uniform and small residual ionization fraction.  